### launching of SPs



In [2]:
from datetime import timedelta
from tqdm import tqdm
import pandas as pd

import pyodbc
from sql.config import SQL_SERVER, DB_NAME
import sql.naming_convention as nc
from sql.sql_requests import SQL_Communicator

In [3]:
def format_time_delta(duration: timedelta):
    # Get the total seconds
    total_seconds = duration.total_seconds()
    # days = duration.days
    hours = duration.seconds // 3600  # Convert seconds to hours
    minutes = (duration.seconds // 60) % 60  # Convert remaining seconds to minutes
    seconds = duration.seconds % 60
    if hours > 0:
        return f'{hours:02d}:{minutes:02d}:{int(seconds):02d}'
    if minutes > 0:
        return f'{minutes:02d}:{int(seconds):02d}'
    return f'{total_seconds:4.2f}'

In [4]:
entity_names =[
    "AddtlCompany_Address",
    "AddtlCompany_BusinessDescription",
    "AddtlCompany_LongBusinessDescription",
    "CompanyBiteSize_CompanyBiteSize",
    "CompanyGeosOfInt_CompanyGeosOfInterest",
    "CompanyIndOfInt_CompanyIndustriesOfInterest",
    "CompanyStagesOfInt_CompanyStagesOfInterest",
    "GICS_GICS",
    "KeyDev_FutureEvent",
    "KeyDev_FutureEventToObjectToEventType",
    "KeyDev_KeyDev",
    "KeyDev_KeyDevToObjectToEventType",
    "Professional_PersonProfessionalCompensation",
    "Professional_PersonProfessional",

    "Symbol_DandBSymbol",
    "FutureEventMkt_FutureEventMkt",
    "FutureEventMkt_FutureEventMktSplitInfo",
    "FutureEventMkt_FutureEventMktToObjectToEventType",

    "KeyDevPlus_KeyDev",
    "KeyDevPlus_KeyDevToObjectToEventType",
    "CompDetail_CompensationAdjustment",
    "CompDetail_CompensationDetail"
]

durations = []
with SQL_Communicator() as sc:
    for en in tqdm(entity_names):
        print(f'flow for target entity {en}..')
        en2 = nc.default_rename(en)
        sp_names = [nc.pull_sp_name(en2), nc.merge_sp_name(en2)]
        ent_durs, flow_td = [en2], timedelta()  
        for sp_name in sp_names:
            exec_stm = f"EXEC {sp_name}"
            print(f'- {exec_stm}...')
            try:
                td = sc.get_execution_metrics(exec_stm)
                ent_durs.append(format_time_delta(td))
                flow_td += td
                print(f'  - duration {format_time_delta(td)}')
            except ex:
                print(f'execution failed {ex}')
        ent_durs.append(format_time_delta(flow_td))
        durations.append(ent_durs)
        # ent_durs.append(sum(ent_durs))
columns = ['Entity name', 'Pull SP', 'Merge SP',  'sum']
df = pd.DataFrame(durations, columns = columns)


connecting...
Connection successful!


  0%|          | 0/22 [00:00<?, ?it/s]

flow for target entity AddtlCompany_Address..
- EXEC dbo.PullData_AddtlCompany_Address2_prc...
duration 01:11
- EXEC dbo.MergeData_AddtlCompany_Address2_prc...


In [8]:
df

,Entity name,Pull SP,Merge SP,sum
0,GICS_GICS2,1.45,0.27,1.71
1,Symbol_GVKeySymbol2,1.49,1.39,2.88
2,Symbol_GVKeyEnhanced2,1.45,1.83,3.28
3,Symbol_DandBSymbol2,37.63,41.30,01:18
4,KeyDev_FutureEvent2,02:39,46.46,03:26
5,KeyDev_FutureEventToObjectToEventType2,28.35,33.26,01:01
6,FutureEventMkt_FutureEventMkt2,02:33,01:12,03:45
7,FutureEventMkt_FutureEventMktSplitInfo2,16.89,18.58,35.47
8,FutureEventMkt_FutureEventMktToObjectToEventType2,42.73,50.40,01:33
